In [1]:
!dir
%cd keras-retinanet

 Datentr�ger in Laufwerk C: ist Windows
 Volumeseriennummer: 3E26-A96E

 Verzeichnis von C:\Users\Yannick\Google Drive\person_detection

03.04.2020  15:55    <DIR>          .
03.04.2020  15:55    <DIR>          ..
31.03.2020  14:54               103 .gitignore
03.04.2020  15:49    <DIR>          .ipynb_checkpoints
02.04.2020  12:39    <DIR>          anchor-optimization
03.04.2020  12:03    <DIR>          custom_models
03.04.2020  14:37            12.282 evaluation.ipynb
31.03.2020  14:54         3.957.556 example_carplates_res50_colab.ipynb
03.04.2020  15:10    <DIR>          keras-retinanet
03.04.2020  15:55            27.265 PersonDetection_res50_widerPerson_colab.ipynb
03.04.2020  15:03            84.536 second_train_notebook.ipynb
03.04.2020  15:55            44.005 tensorboard_executer.ipynb
01.04.2020  16:22    <DIR>          WiderPerson
               6 Datei(en),      4.125.747 Bytes
               7 Verzeichnis(se), 52.877.987.840 Bytes frei
C:\Users\Yannick\Google Drive\perso

In [4]:
!pip install .
!python setup.py build_ext --inplace


Processing c:\users\yannick\google drive\person_detection\keras-retinanet

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Yannick\Anaconda3\envs\tensorflow\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Yannick\\AppData\\Local\\Temp\\pip-req-build-mhybcqkx\\setup.py'"'"'; __file__='"'"'C:\\Users\\Yannick\\AppData\\Local\\Temp\\pip-req-build-mhybcqkx\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Yannick\AppData\Local\Temp\pip-wheel-p1lsdypu'
       cwd: C:\Users\Yannick\AppData\Local\Temp\pip-req-build-mhybcqkx\
  Complete output (90 lines):
  running bdist_wheel
  running build
  running build_py
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\keras_retinanet
  copying keras_retinanet\initializers.py -> build\lib.win-amd64-3.7\keras_retinanet
  copying keras_retinanet\losses.py -> build\lib.win-amd64-3.7\keras_retinanet
  copy


  Running setup.py clean for keras-retinanet
Failed to build keras-retinanet
    Running setup.py install for keras-retinanet: started
    Running setup.py install for keras-retinanet: finished with status 'error'


    copying keras_retinanet\utils\__init__.py -> build\lib.win-amd64-3.7\keras_retinanet\utils
    creating build\lib.win-amd64-3.7\tests\backend
    copying tests\backend\test_common.py -> build\lib.win-amd64-3.7\tests\backend
    copying tests\backend\__init__.py -> build\lib.win-amd64-3.7\tests\backend
    creating build\lib.win-amd64-3.7\tests\layers
    copying tests\layers\test_filter_detections.py -> build\lib.win-amd64-3.7\tests\layers
    copying tests\layers\test_misc.py -> build\lib.win-amd64-3.7\tests\layers
    copying tests\layers\__init__.py -> build\lib.win-amd64-3.7\tests\layers
    creating build\lib.win-amd64-3.7\tests\models
    copying tests\models\test_densenet.py -> build\lib.win-amd64-3.7\tests\models
    copying tests\models\test_mobilenet.py -> build\lib.win-amd64-3.7\tests\models
    copying tests\models\__init__.py -> build\lib.win-amd64-3.7\tests\models
    creating build\lib.win-amd64-3.7\tests\preprocessing
    copying tests\preprocessing\test_csv_generat

running build_ext
skipping 'keras_retinanet/utils\compute_overlap.c' Cython extension (up-to-date)
building 'keras_retinanet.utils.compute_overlap' extension
C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\BIN\x86_amd64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Users\Yannick\Anaconda3\envs\tensorflow\include -IC:\Users\Yannick\Anaconda3\envs\tensorflow\include -IC:\Users\Yannick\Anaconda3\envs\tensorflow\lib\site-packages\numpy\core\include "-IC:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\INCLUDE" "-IC:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\ATLMFC\INCLUDE" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.10240.0\ucrt" /Tckeras_retinanet/utils\compute_overlap.c /Fobuild\temp.win-amd64-3.7\Release\keras_retinanet/utils\compute_overlap.obj
compute_overlap.c
c:\users\yannick\anaconda3\envs\tensorflow\include\pyconfig.h(203): fatal error C1083: Datei (Include) kann nicht ge�ffnet werden: "basetsd.h": No such file or directory


error: command 'C:\\Program Files (x86)\\Microsoft Visual Studio 14.0\\VC\\BIN\\x86_amd64\\cl.exe' failed with exit status 2


In [9]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import urllib
import os
import csv
import cv2
import time
from PIL import Image
from IPython.core.debugger import set_trace
import shutil

import pdb

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)


In [ ]:
annot_path = '/content/drive/My Drive/person_detection/WiderPerson/Annotations'
images_path = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_csv_path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations_new.csv'
keras_path = '/content/drive/My Drive/person_detection/keras-retinanet'
#Remove classes which shall be left out in the csv
classes_ids = {1:'pedestrian',
               2:'riders',
               3:'partially-visible',
               5:'crowd'}

# Training



In [ ]:
#TODO: Make it also work for bigger architecture:"/fizyr/keras-retinanet/releases/download/0.5.1/resnet152_oid_v1.0.0.h5"

In [ ]:
PRETRAINED_MODEL_VANILLA = './snapshots/_pretrained_model.h5'
PRETRAINED_MODEL_WI = '../custom_models/resnet50_e15_b8_s1000_train_0304.h5'
  

In [ ]:
%cd /content/drive/My Drive/person_detection/keras-retinanet
!python keras_retinanet/bin/train.py  --image-min-side=400 \
                                      --image-max-side=1000 \
                                      --tensorboard-dir '' \
                                      --freeze-backbone \
                                      --weights {PRETRAINED_MODEL_VANILLA} \
                                      --lr 1e-11 \
                                      --batch-size 8 \
                                      --steps 1000 \
                                      --epochs 10 \
                                      csv annot_train_filtered_450_2000.csv classes.csv \
                                      #'--imagenet-weights'
                                      # --snapshot {PRETRAINED_MODEL_VANILLA} \
                                    # --val-annotations annot_test.csv \
                                    # --compute-val-loss \
                                    # --random-transform \

%cd ../../../../../

## Debugging

In [ ]:
%cd /content/drive/My Drive/person_detection/keras-retinanet
!python keras_retinanet/bin/debug.py --annotations \
                                     --anchors \
                                     --no-gui \
                                     csv annot_train.csv classes.csv
%cd ../../../../..

/content/drive/My Drive/person_detection/keras-retinanet
Using TensorFlow backend.
2020-04-03 11:29:23.188330: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-04-03 11:29:23.188459: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2020-04-03 11:29:23.188480: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/


### Anchor Optimization

In [ ]:
%cd /content/drive/My Drive/person_detection
!git clone https://github.com/martinzlocha/anchor-optimization.git
%cd anchor-optimization
!pip install .
!python setup.py build_ext --inplace

In [ ]:
!python ./../anchor-optimization/optimize_anchors.py annot_train.csv


## Functions


In [ ]:
# Generate the classes csv file
with open(annotations, newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for filename in os.listdir(images_path)[0:10]:
    if str(filename + '.txt') in annot:
      f = open(annot_path + '/' +  filename + '.txt', 'r')
      
      for index, line in enumerate(f): 
        if index == 0: 
          if line.strip() == '0':
            print('Not any  object in the image!')
          continue
          
        else: 
          split_line = line[:line.find('/')].split(' ')
          first_char = split_line.pop(0)
          split_line.insert(len(split_line), first_char)
          split_line.insert(0, images_path + '/' + filename)
          #convert from index to class label 
          try:
            split_line[-1] = classes_ids[int(split_line[-1])]
          except: 
            continue
          split_line[0] = split_line[0]

          csv_writer.writerow(split_line)
      
      f.close()

#create indexing csv file
with open('classes.csv', newline='', mode='x') as csvfile:
  csv_writer = csv.writer(csvfile, delimiter=',')
  for index, key in enumerate(classes_ids.keys()):
    csv_writer.writerow([str(classes_ids[key]), index])


#Check for wrongly annotaded bounding boxes
path = '/content/drive/My Drive/person_detection/keras-retinanet/annotations.csv'
def check_bb(path):
  colnames = ['filename', 'x1', 'y1', 'x2', 'y2', 'class_label']
  df = pd.read_csv(path, names=colnames)
  df_new = df.loc[(df.x1 < df.x2) & (df.y1 < df.y2), : ]
  print ('Reduces shape from ', df.shape, 'to ', df_new.shape)
  return df_new





In [ ]:
def replace_annoation_folder(new_folder, annot_path, old_img_path):
  '''Start this function from folder root, otherwise wont work properly'''

  df_annot = pd.read_csv(annot_path + '/annotations_pathkeras.csv',
                         names=['name', 'x1', 'y1', 'x2', 'y2', 'label'])
  df_annot = df_annot.reset_index().drop(0).drop(columns='index')
  df_annot['name'] = df_annot['name'].str.replace(old_img_path, new_folder)
  df_annot = df_annot.dropna()
  df_annot.to_csv(annot_path + '/annotations_new.csv',
                  index=None, header=None)
  return


old_folder = '/content/drive/My Drive/PersonDetection/WiderPerson/Images'
new_folder = '/content/drive/My Drive/person_detection/WiderPerson/Images'
annot_path_csv = '/content/drive/My Drive/person_detection/keras-retinanet'
replace_annoation_folder(new_folder, annot_path_csv, old_folder)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
#Splitting into train and test data

train_df, test_df = train_test_split(
  pd.read_csv(annot_csv_path, header=None), 
  test_size=0.15, 
  random_state=RANDOM_SEED
)

train_df.to_csv(keras_path + '/annot_train.csv', header=None, index=None)
test_df.to_csv(keras_path + '/annot_test.csv', header=None, index=None)

/content/drive/My Drive/person_detection/WiderPerson/Annotations
